# OpenAI - A partir de 1 product_id o programa gera os 3 produtos mais semelhantes da Amazon

In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_distances

# 1) Carrega o DataFrame com embeddings gerados
df = pd.read_csv('OpenAI_embeddings.csv')

# 2) Colunas de embedding a serem combinadas
emb_cols = [
    'embedding_product_name',
    'embedding_category',
    'embedding_about_product',
    'embedding_review_title',
    'embedding_review_content'
]

# 3) Converte cada embedding de string para np.array e concatena
def combine_embeddings(row):
    parts = []
    for col in emb_cols:
        emb_str = row[col]
        if isinstance(emb_str, str):
            emb = np.array(ast.literal_eval(emb_str), dtype=float)
            parts.append(emb)
    return np.concatenate(parts)

# Aplica a combinação e empilha numa matriz
df['emb_combined'] = df.apply(combine_embeddings, axis=1)
emb_matrix = np.vstack(df['emb_combined'].values)

# 4) Cria mapeamento product_id → índice no DataFrame
id_to_idx = {pid: idx for idx, pid in enumerate(df['product_id'].values)}

# 5) Função para buscar os top_k produtos mais similares
def find_similar_products(query_id, top_k=3):
    if query_id not in id_to_idx:
        raise KeyError(f"product_id {query_id} não encontrado.")
    idx = id_to_idx[query_id]
    q_emb = emb_matrix[idx].reshape(1, -1)
    dists = cosine_distances(q_emb, emb_matrix)[0]
    dists[idx] = np.inf  # ignora o próprio item
    nearest = np.argsort(dists)[:top_k]
    return df.iloc[nearest][['product_id', 'product_name']].reset_index(drop=True)


In [7]:
# 6) Uso
consulta = "B07JW9H4J1"  # substitua pelo product_id desejado
top3 = find_similar_products(consulta, top_k=3)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 200)
print(top3)

   product_id                                                                                                                                                        product_name
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)
1  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)
2  B07JW1Y6XV                                            Wayona Nylon Braided 3A Lightning to USB A Syncing and Fast Charging Data Cable for iPhone, Ipad (3 FT Pack of 1, Black)
